# Marqo

This notebook shows how to use functionality related to the Marqo database.

In [1]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Marqo
from langchain.document_loaders import TextLoader

In [2]:
from langchain.document_loaders import TextLoader
loader = TextLoader('../../../state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [3]:
import marqo 

# initialize marqo
marqo_url = "http://localhost:8882" # if using marqo cloud replace with your endpoint (console.marqo.ai)
marqo_api_key = "" # if using marqo cloud replace with your api key (console.marqo.ai)

client = marqo.Client(url=marqo_url, api_key=marqo_api_key)

index_name = "langchain-demo"

docsearch = Marqo.from_documents(docs, index_name=index_name)

query = "What did the president say about Ketanji Brown Jackson"
result_docs = docsearch.similarity_search(query)

Index langchain-demo exists.


In [4]:
print(result_docs[0].page_content)

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


In [5]:
result_docs = docsearch.similarity_search_with_score(query)
print(result_docs[0][0].page_content, result_docs[0][1], sep="\n")

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
0.68647254


## Additional features

One of the powerful features of Marqo as a vectorstore is that you can use indexes created externally. For example:

+ If you had a database of image and text pairs from another application, you can simply just use it in langchain with the Marqo vectorstore. Note that bringing your own multimodal indexes will disable the `add_texts` method.

+ If you had a database of text documents, you can bring it into the langchain framework and add more texts through `add_texts`.

The documents that are returned are customised by passing your own function to the `page_content_builder` callback in the search methods.

#### Multimodal Example

In [6]:

# use a new index
index_name = "langchain-multimodal-demo"

# incase the demo is re-run
try:
    client.delete_index(index_name)
except Exception:
    print(f"Creating {index_name}")
    
# This index could have been created by another system
settings = {"treat_urls_and_pointers_as_images": True, "model": "ViT-L/14"}
client.create_index(index_name, **settings)
client.index(index_name).add_documents(
    [   
        # image of a bus
        {
            "caption": "Bus",
            "image": "https://raw.githubusercontent.com/marqo-ai/marqo/mainline/examples/ImageSearchGuide/data/image4.jpg"
        },
        # image of a plane
        {   
            "caption": "Plane", 
            "image": "https://raw.githubusercontent.com/marqo-ai/marqo/mainline/examples/ImageSearchGuide/data/image2.jpg"
        }
    ],
)

{'errors': False,
 'processingTimeMs': 4675.6921890009835,
 'index_name': 'langchain-multimodal-demo',
 'items': [{'_id': '7af25f35-5d41-4ff5-95fa-ab6bd6755176',
   'result': 'created',
   'status': 201},
  {'_id': '70434d17-2680-4e33-b060-a37b9b8b6959',
   'result': 'created',
   'status': 201}]}

In [7]:
def get_content(res):
    """Helper to format Marqo's documents into text to be used as page_content"""
    return f"{res['caption']}: {res['image']}"

docsearch = Marqo(client, index_name, page_content_builder=get_content)


query = "vehicles that fly"
doc_results = docsearch.similarity_search(query)

In [8]:
for doc in doc_results:
    print(doc.page_content)

Plane: https://raw.githubusercontent.com/marqo-ai/marqo/mainline/examples/ImageSearchGuide/data/image2.jpg
Bus: https://raw.githubusercontent.com/marqo-ai/marqo/mainline/examples/ImageSearchGuide/data/image4.jpg


#### Text only example

In [9]:

# use a new index
index_name = "langchain-byo-index-demo"

# incase the demo is re-run
try:
    client.delete_index(index_name)
except Exception:
    print(f"Creating {index_name}")

# This index could have been created by another system
client.index(index_name).add_documents(
    [   
        {
            "Title": "Smartphone",
            "Description": "A smartphone is a portable computer device that combines mobile telephone "
            "functions and computing functions into one unit.",
        },
        {   
            "Title": "Telephone",
            "Description": "A telephone is a telecommunications device that permits two or more users to"
            "conduct a conversation when they are too far apart to be easily heard directly.",
        }
    ],
)

{'errors': False,
 'processingTimeMs': 500.1302719992964,
 'index_name': 'langchain-byo-index-demo',
 'items': [{'_id': 'cbad6f9e-a4ea-45c6-9a85-1b9c0a59827c',
   'result': 'created',
   'status': 201},
  {'_id': 'c0be68cb-8847-4e95-a4c9-4791b54f772c',
   'result': 'created',
   'status': 201}]}

In [10]:
# Note text indexes retain the ability to use add_texts despite different field names in documents
# this is because the page_content_builder callback lets you handle these document fields as required

def get_content(res):
    """Helper to format Marqo's documents into text to be used as page_content"""
    if 'text' in res:
        return res['text']
    return res['Description']


docsearch = Marqo(client, index_name, page_content_builder=get_content)

docsearch.add_texts(["This is a document that is about elephants"])


['484d8436-cb09-49f2-8f9d-39671c7ebfaa']

In [11]:
query = "modern communications devices"
doc_results = docsearch.similarity_search(query)

print(doc_results[0].page_content)

A smartphone is a portable computer device that combines mobile telephone functions and computing functions into one unit.


In [12]:
query = "elephants"
doc_results = docsearch.similarity_search(query, page_content_builder=get_content)

print(doc_results[0].page_content)

This is a document that is about elephants


## Weighted Queries

We also expose marqos weighted queries which are a powerful way to compose complex semantic searches.

In [13]:
query = {"communications devices" : 1.0}
doc_results = docsearch.similarity_search(query)
print(doc_results[0].page_content)

A smartphone is a portable computer device that combines mobile telephone functions and computing functions into one unit.


In [14]:
query = {"communications devices" : 1.0, "technology post 2000": -1.0}
doc_results = docsearch.similarity_search(query)
print(doc_results[0].page_content)

A telephone is a telecommunications device that permits two or more users toconduct a conversation when they are too far apart to be easily heard directly.
